# 🤖 AI Spreadsheet Assistant

A free AI-powered tool for automating Excel/CSV tasks using natural language.

**Features:**
- 📊 Excel/CSV file support
- 🆓 Free Hugging Face API integration
- 📝 Natural language instructions
- 🔧 Step-by-step guides with formulas

In [ ]:
# Install required packages
!pip install pandas openpyxl requests ipywidgets
print("🎉 Packages installed successfully!")

In [ ]:
import pandas as pd
import requests
import json
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown, HTML
from google.colab import files
import io
import warnings
warnings.filterwarnings('ignore')

print("📚 Libraries imported successfully!")

In [ ]:
class AIModelHandler:
    """Handles AI model interactions - FREE Hugging Face support"""
    
    def __init__(self, api_key: str, model_type: str = "huggingface"):
        self.api_key = api_key
        self.model_type = model_type
        
        if model_type == "huggingface":
            self.hf_headers = {"Authorization": f"Bearer {api_key}"}
            self.hf_url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1"

    def generate_instructions(self, task_description: str, data_preview: str) -> str:
        """Generate step-by-step instructions using FREE Hugging Face API"""
        
        prompt = f"""Create detailed Excel instructions for: {task_description}

Data preview: {data_preview}

Provide:
1. Manual steps for Excel/Google Sheets
2. Exact formulas with code
3. Expected results

Format as numbered steps."""

        try:
            if self.model_type == "huggingface":
                payload = {
                    "inputs": prompt,
                    "parameters": {
                        "max_new_tokens": 800,
                        "temperature": 0.3,
                        "return_full_text": False
                    }
                }
                
                response = requests.post(self.hf_url, headers=self.hf_headers, json=payload)
                
                if response.status_code == 200:
                    result = response.json()
                    if isinstance(result, list) and len(result) > 0:
                        generated_text = result[0].get('generated_text', 'No response generated')
                        return f"## 📋 Step-by-Step Instructions\n\n{generated_text}"
                    return f"## 📋 AI Response\n\n{str(result)}"
                else:
                    return f"## ❌ Error\n\nAPI Status: {response.status_code}. Try again in 30 seconds."
                    
        except Exception as e:
            return f"## ❌ Error\n\nSomething went wrong: {str(e)}"

print("🆓 FREE Hugging Face AI Handler ready!")

In [ ]:
class SpreadsheetAIAssistant:
    """Main AI Assistant for spreadsheet automation"""
    
    def __init__(self):
        self.ai_handler = None
        self.current_data = None
        self.filename = None

    def setup_ai(self, api_key: str, model_type: str):
        """Initialize AI model"""
        try:
            self.ai_handler = AIModelHandler(api_key, model_type)
            print(f"✅ {model_type.upper()} AI model initialized!")
            return True
        except Exception as e:
            print(f"❌ Error initializing AI model: {e}")
            return False

    def load_data_from_file(self, file_content: bytes, filename: str) -> bool:
        """Load data from uploaded file"""
        try:
            if filename.endswith('.csv'):
                self.current_data = pd.read_csv(io.BytesIO(file_content))
            else:
                self.current_data = pd.read_excel(io.BytesIO(file_content))
            
            self.filename = filename
            
            if self.current_data is not None and not self.current_data.empty:
                print(f"✅ Data loaded successfully!")
                print(f"📊 Shape: {self.current_data.shape}")
                print(f"📂 File: {filename}")
                return True
            else:
                print("❌ Failed to load data - file appears to be empty")
                return False
        except Exception as e:
            print(f"❌ Error loading data: {e}")
            return False

    def get_data_preview(self, n_rows: int = 10) -> str:
        """Get a preview of the current data for AI analysis"""
        if self.current_data is None:
            return "No data loaded"
        
        preview = f"Data Shape: {self.current_data.shape}\n"
        preview += f"Columns: {list(self.current_data.columns)}\n\n"
        preview += f"First {n_rows} rows:\n{self.current_data.head(n_rows).to_string()}"
        return preview

    def display_data_preview(self, n_rows: int = 10):
        """Display formatted data preview"""
        if self.current_data is None:
            display(Markdown("**❌ No data loaded**"))
            return
        
        summary_md = f"""
## 📊 Data Summary
- **Rows:** {self.current_data.shape[0]:,}
- **Columns:** {self.current_data.shape[1]}
- **File:** {self.filename}

### Column Names:
{', '.join([f'`{col}`' for col in self.current_data.columns])}
"""
        
        display(Markdown(summary_md))
        
        preview_md = f"## 👀 First {n_rows} Rows Preview\n\n"
        df_preview = self.current_data.head(n_rows)
        preview_md += df_preview.to_markdown(index=False)
        
        display(Markdown(preview_md))

    def generate_solution(self, instructions: str) -> str:
        """Generate AI-powered solution for spreadsheet tasks"""
        if not self.ai_handler:
            return "## ❌ Error\n\nAI model not initialized. Please set up API key first."
        
        if self.current_data is None:
            return "## ❌ Error\n\nNo data loaded. Please load a spreadsheet first."
        
        print("🧠 Analyzing your data and generating solution...")
        data_preview = self.get_data_preview()
        solution = self.ai_handler.generate_instructions(instructions, data_preview)
        
        return solution

# Initialize the assistant
assistant = SpreadsheetAIAssistant()
print("🤖 AI Spreadsheet Assistant initialized!")

In [ ]:
# Create UI widgets
display(HTML("<h1>🤖 AI Spreadsheet Assistant</h1>"))
display(HTML("<p>Upload your Excel/CSV file and describe what you want to do!</p>"))

# File upload widget
file_upload = widgets.FileUpload(
    accept='.xlsx,.xls,.csv',
    multiple=False,
    description='Upload File'
)

# Instructions input
instruction_input = widgets.Textarea(
    placeholder='Example: "Remove duplicates by email, format phone numbers as (XXX) XXX-XXXX"',
    description='Instructions:',
    layout=widgets.Layout(width='90%', height='100px')
)

# API configuration
api_key_input = widgets.Password(
    placeholder='Enter your Hugging Face API key (hf_...)',
    description='API Key:',
    layout=widgets.Layout(width='50%')
)

model_selector = widgets.Dropdown(
    options=[('🆓 Hugging Face (FREE)', 'huggingface')],
    description='AI Model:',
    value='huggingface'
)

# Buttons
load_file_button = widgets.Button(
    description='📁 Load Excel/CSV File',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

generate_button = widgets.Button(
    description='🚀 Generate Instructions',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

# Output areas
data_output = widgets.Output()
solution_output = widgets.Output()

print("🎨 User interface created!")

In [ ]:
def on_load_file_click(b):
    """Handle file upload"""
    with data_output:
        clear_output()
        
        if not file_upload.value:
            print("❌ Please upload a file first.")
            return
        
        # Get the uploaded file
        uploaded_file = list(file_upload.value.values())[0]
        file_content = uploaded_file['content']
        filename = uploaded_file['metadata']['name']
        
        print(f"📁 Loading file: {filename}")
        
        # Load the data
        success = assistant.load_data_from_file(file_content, filename)
        
        if success:
            print("\n" + "="*50)
            assistant.display_data_preview(10)

def on_generate_click(b):
    """Handle instruction generation"""
    with solution_output:
        clear_output()
        
        # Validate inputs
        if not instruction_input.value.strip():
            print("❌ Please enter instructions for what you want to do.")
            return
        
        if not api_key_input.value.strip():
            print("❌ Please enter your Hugging Face API key.")
            return
        
        # Setup AI model
        success = assistant.setup_ai(api_key_input.value, 'huggingface')
        if not success:
            return
        
        # Generate solution
        solution_md = assistant.generate_solution(instruction_input.value)
        display(Markdown(solution_md))

# Connect event handlers
load_file_button.on_click(on_load_file_click)
generate_button.on_click(on_generate_click)

print("🔗 Event handlers connected!")

In [ ]:
# Display the complete interface
display(HTML("<hr><h2>🔑 Step 1: Enter Your Hugging Face API Key</h2>"))
display(widgets.HBox([api_key_input, model_selector]))

display(HTML("<h2>📁 Step 2: Upload Your Excel/CSV File</h2>"))
display(widgets.HBox([file_upload, load_file_button]))

display(HTML("<h2>👀 Data Preview</h2>"))
display(data_output)

display(HTML("<hr><h2>💭 Step 3: Describe Your Task</h2>"))
display(instruction_input)
display(generate_button)

display(HTML("<h2>📋 Generated Instructions</h2>"))
display(solution_output)

# Instructions for users
instructions_html = """
<div style="background-color: #f0f9ff; padding: 20px; border-radius: 10px; margin-top: 20px;">
<h3>🚀 How to Get Your FREE API Key:</h3>
<ol>
<li>Visit <a href="https://huggingface.co/join" target="_blank">Hugging Face (FREE signup)</a></li>
<li>Go to Settings → Access Tokens</li>
<li>Create new token (select "Read" permissions)</li>
<li>Copy token starting with "hf_"</li>
</ol>

<h3>💡 Example Tasks:</h3>
<ul>
<li>"Remove duplicate rows based on email column"</li>
<li>"Format phone numbers to (XXX) XXX-XXXX style"</li>
<li>"Split full name column into first and last name"</li>
<li>"Convert dates to MM/DD/YYYY format"</li>
</ul>
</div>
"""

display(HTML(instructions_html))

print("🎉 AI Spreadsheet Assistant is ready to use!")